In [ ]:
import os
from datetime import datetime, timedelta
import ipywidgets as widgets
import plotly.graph_objs as go
import yfinance as yf
import pandas as pd
from IPython.display import display

In [ ]:
category = ["Financial", "Indices", "Commodities"]
description = ["Euro-dollar", "Two-Year US Treasury Note", "Five-Year US Treasury Note"]
chart_type = ["Historical Time Series", "Future Curve"]

In [ ]:
def big_num(num):
    if num > 1_000_000_000_000:
        return f"{round(num/1_000_000_000_000,2)}T"
    if num > 1_000_000_000:
        return f"{round(num/1_000_000_000,2)}B"
    if num > 1_000_000:
        return f"{round(num/1_000_000,2)}M"
    if num > 1_000:
        return f"{num/round(1_000,2)}K"
    return f"{round(num,2)}"


def clean_str(string):
    new_str = ""
    for letter in string:
        if letter.isupper():
            new_str += " "
        new_str += letter
    return new_str.title()


def format_plotly(fig, data, chart, calc=None):
    fig.update_yaxes(title=None)
    fig.update_xaxes(title=None)
    if calc:
        if len(calc) == 1:
            fig_title = f"{calc[0]} of {data}"
        else:
            fig_title = f"{', '.join(calc)} of {data}"
    else:
        fig_title = "Volume"
    height = 500 if chart == "main" else 300
    fig.update_layout(
        margin=dict(l=0, r=10, t=10, b=10),
        autosize=False,
        width=900,
        height=height,
        legend=dict(orientation="h"),
        title={
            "text": fig_title,
            "y": 0.95,
            "x": 0.5,
            "xanchor": "center",
            "yanchor": "top",
        },
    )


def create_line(visual, x, y, name, data, fig):
    if visual == "line":
        plot = go.Scatter(x=x, y=y[data], mode="lines", name=name, connectgaps=True)
    if visual == "scatter":
        plot = go.Scatter(x=x, y=y[data], mode="markers", name=name)
    if visual == "candle":
        plot = go.Candlestick(
            x=x,
            open=y["Open"],
            close=y["Close"],
            high=y["High"],
            low=y["Low"],
            name=name,
        )
    fig.add_trace(plot)


def show_fig(fig):
    config = {"showTips": False, "scrollZoom": True}
    if os.environ.get("SERVER_SOFTWARE", "jupyter").startswith("voila"):
        fig.show(config=config, renderer="notebook")
    else:
        fig.show(config=config)


def table_data(infos):
    cols = ["Ticker"] + list(infos)
    data = pd.DataFrame(columns=cols)
    data["Ticker"] = [clean_row[x] for x in rows]
    for ticker in list(infos):
        data[ticker] = [clean_data[x](infos[ticker].get(x, None)) for x in rows]
    new_cols = {k: clean_str(k) for k in rows}
    return data

In [ ]:
class Chart:
    def __init__(self):
        self.last_tickers = ""
        self.df = pd.DataFrame()
        self.infos = {}

    def create_stock(self, category, description, chart_type, contracts, tickers):
        print(category)
        print(description)
        print(chart_type)
        if tickers != self.last_tickers:
            interval = "1d"
            self.df = yf.download(
                tickers, period="max", interval=interval, progress=False
            )
            self.last_tickers = tickers

        start_n = datetime(2020,1,1)
        end_n = datetime(2022, 1, 1)
        fig = go.Figure()
        calcs = self.df
        result = calcs.loc[(calcs.index >= start_n) & (calcs.index <= end_n)]

        if len(result.columns) == 6:
            name = f"{tickers.split(',')[0]}"
            create_line("line", result.index, result, name, "Close", fig)

        else:
            for val in result.columns.levels[1]:
                vals = result.xs(val, axis=1, level=1, drop_level=True)
                name = f"{val.upper()}"
                create_line("line", result.index, vals, name, "Close", fig)

        format_plotly(fig, "Close", "main", "raw")
        show_fig(fig)

In [ ]:
w_auto = widgets.Layout(width="auto")

cat_widget = widgets.SelectMultiple(options=category, value=[category[0]], layout=w_auto)
desc_widget = widgets.SelectMultiple(options=description, value=[description[0]], layout=w_auto)
chart_widget = widgets.SelectMultiple(options=chart_type, value=[chart_type[0]], layout=w_auto)

contracts_widget = widgets.Dropdown(options=list(range(1, 13)), value=6, layout=w_auto, description="Contracts")

tickers_widget = widgets.Text(
    value="TSLA"#, layout=widgets.Layout(width="auto", height="100%")
)
controls = widgets.HBox(
    [tickers_widget, cat_widget, desc_widget, chart_widget, contracts_widget],
    layout=widgets.Layout(width="90%"),
)
chart = Chart()
stocks_view = widgets.interactive_output(
    chart.create_stock,
    {
        "category": cat_widget,
        "description": desc_widget,
        "chart_type": chart_widget,
        "contracts": contracts_widget,
        "tickers": tickers_widget,
    },
)

title_html = "<h1>Stock Analysis Dashboard</h1>"

app_contents = [widgets.HTML(title_html), controls, stocks_view]
app = widgets.VBox(app_contents)
display(app)